这节课，我们来探究一下什么是转置卷积

转置卷积其实是一种特殊的卷积过程，这个转置卷积也是2016年才被提出来的
* 一般来说，在卷积操作中，我们的原始图像到feature map大小都是在不断缩小的，当然通过padding可以增大，但是padding这种方法想也知道相当不可靠（大量padding的话）
* 传统的卷积在图像分类和物体识别里面是可以的，因为我需要最终预测的其实都只是整个图像中的一小部分像素
* 但是到了语义分割中，我要对每个像素的标号都去进行预测，如果我再用传统卷积去不断缩小我的图像大小，那我最后预测个屁呢
* 像自动驾驶这种语义分割的常用场景，我但凡几个像素预测错了，说实话都有可能是惨绝人寰的车祸

转置卷积的具体操作其实有点难以用语言来描述，详情请看李沐老师视频里面的讲解，总的来说有以下几点
1. 转置卷积的输入输出大小实际上是和传统卷积可以看作是一种转置反过来的关系（可能这也是名字的由来？）
2. 转置卷积的卷积核，是针对输入的每个像素去做一个按元素计算的，所以会达到一个增大大小的作用
3. 而在转置卷积中，padding/stride这两个操作同样都是反着来的，都是作用在输出上的
4. 所以同样超参数的卷积和转置卷积，是完全一样的，只是反过来

In [1]:
import torch
from torch import nn
from d2l import torch as d2l

In [2]:
def trans_conv(X, K):
    h, w = K.shape
    Y = torch.zeros((X.shape[0] + h - 1, X.shape[1] + w - 1))
    for i in range(X.shape[0]):
        for j in range(X.shape[1]):
            Y[i: i + h, j: j + w] += X[i, j] * K
    return Y

In [3]:
# 上面我们实现的就是一个最最简单的转置卷积操作了，没考虑padding、stride
X = torch.tensor([[0.0, 1.0], [2.0, 3.0]])
K = torch.tensor([[0.0, 1.0], [2.0, 3.0]])
trans_conv(X, K)

tensor([[ 0.,  0.,  1.],
        [ 0.,  4.,  6.],
        [ 4., 12.,  9.]])

In [4]:
# 更加复杂的输入数据，我们通过torch来实现
X, K = X.reshape(1, 1, 2, 2), K.reshape(1, 1, 2, 2)
tconv = nn.ConvTranspose2d(1, 1, kernel_size=2, bias=False)
tconv.weight.data = K
tconv(X)

tensor([[[[ 0.,  0.,  1.],
          [ 0.,  4.,  6.],
          [ 4., 12.,  9.]]]], grad_fn=<ConvolutionBackward0>)

In [5]:
X, K = X.reshape(1, 1, 2, 2), K.reshape(1, 1, 2, 2)
tconv = nn.ConvTranspose2d(1, 1, kernel_size=2, bias=False)
tconv.weight.data = K
tconv(X)

tensor([[[[ 0.,  0.,  1.],
          [ 0.,  4.,  6.],
          [ 4., 12.,  9.]]]], grad_fn=<ConvolutionBackward0>)

下面这里我们会发现，这里的填充和我们理解的好像不太一样？怎么加了之后反而变成一个4了呢？
* 其实转置卷积里面的填充，确实会让输出减小
* 啥意思呢？转置卷积的填充作用在输出上，而我们这里指定的padding其实意思是有多少输出是padding得来的
* 也就是说真实的输出，是需要去掉我们这里定义的padding的，那我们这里原始输出是一个33矩阵，设置padding为1，自然就只剩下最中间的4了

In [6]:
tconv = nn.ConvTranspose2d(1, 1, kernel_size=2, padding=1, bias=False)
tconv.weight.data = K
tconv(X)

tensor([[[[4.]]]], grad_fn=<ConvolutionBackward0>)

stride则是和我们想象的一样，会让输出变大

In [7]:
tconv = nn.ConvTranspose2d(1, 1, kernel_size=2, stride=2, bias=False)
tconv.weight.data = K
tconv(X)

tensor([[[[0., 0., 0., 1.],
          [0., 0., 2., 3.],
          [0., 2., 0., 3.],
          [4., 6., 6., 9.]]]], grad_fn=<ConvolutionBackward0>)

In [8]:
# 这段代码是在证明，我们前面说的完全反过来的操作是正确的
X = torch.rand(size=(1, 10, 16, 16))
conv = nn.Conv2d(10, 20, kernel_size=5, padding=2, stride=3)
tconv = nn.ConvTranspose2d(20, 10, kernel_size=5, padding=2, stride=3)
tconv(conv(X)).shape == X.shape

True

下面这两部分是我们前面提到的，卷积操作其实就是一种特殊的矩阵乘法，所以我们这里把卷积核转化为矩阵，来观察其中的数据变化情况

In [9]:
X = torch.arange(9.0).reshape(3, 3)
K = torch.tensor([[1.0, 2.0], [3.0, 4.0]])
Y = d2l.corr2d(X, K)
Y

tensor([[27., 37.],
        [57., 67.]])

In [10]:
def kernel2matrix(K):
    k, W = torch.zeros(5), torch.zeros((4, 9))
    k[:2], k[3:5] = K[0, :], K[1, :]
    W[0, :5], W[1, 1:6], W[2, 3:8], W[3, 4:] = k, k, k, k
    return W

W = kernel2matrix(K)
W

tensor([[1., 2., 0., 3., 4., 0., 0., 0., 0.],
        [0., 1., 2., 0., 3., 4., 0., 0., 0.],
        [0., 0., 0., 1., 2., 0., 3., 4., 0.],
        [0., 0., 0., 0., 1., 2., 0., 3., 4.]])

In [11]:
Y == torch.matmul(W, X.reshape(-1)).reshape(2, 2)

tensor([[True, True],
        [True, True]])

In [12]:
Z = trans_conv(Y, K)
Z == torch.matmul(W.T, Y.reshape(-1)).reshape(3, 3)

tensor([[True, True, True],
        [True, True, True],
        [True, True, True]])

* 从意义上来说，转置卷积可以被认为，知道一个像素是猫，转置卷积将猫整体恢复出来
* 转置卷积的真正意义是，大小恢复到你的输入，但是里面的值可以是不一样的（当然你可以学成完全恢复）
* 数学上有反卷积这个定义，反卷积是卷积操作子的严格逆运算
* 但是dl里面的转置卷积或者反卷积，其实本质上还是卷积网络，只是做了转置卷积操作（形状回去了，数值不一定）